In [ ]:
# Setting up DMSP algorithms params

# Loading the denoiser network

# Limit the GPU access
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
# Loading the model
DAE = tf.saved_model.load('DAE')

# setting DMSP parameterss
params = {}
params['denoiser'] = DAE
params['sigma_dae'] = 11.0
params['num_iter'] = 300
params['mu'] = 0.9
params['alpha'] = 0.1


In [ ]:
# Setting dataset information

from os import walk
from os.path import join

dataset_name = 'Set14'
data_folder = 'data/Benchmark datasets/' + dataset_name

file_paths = []
for (dirpath, dirnames, filenames) in walk(data_folder):
    file_paths.extend(filenames)
    break

In [ ]:
import numpy as np
from DMSPRestore import filter_image
from PIL import Image

from DMSPRestore import pad_image
from DMSPRestore import DMSP_restore, pad_image_rm

k_nanmes = ['k_1', 'k_2', 'k_3', 'k_4', 'k_5', 'k_6', 'k_7', 'k_8']

In [ ]:
# running experiments

for downsampling_rate in [2,4]:
    for sigma_d in [0.0]:
        for kernel_name in k_nanmes:
            # kernel size seems to be set to 15x15 in the experiments
            kernel = np.array(Image.open('data/'+kernel_name+'.png'), dtype='float32')[5:-5,5:-5,0]
            kernel = kernel / np.sum(kernel[:])

            out_folder = 'data/results/' + dataset_name + '_' + kernel_name + '_' + str(downsampling_rate) + '_' + str(sigma_d)
            if not os.path.exists(out_folder):
                os.makedirs(out_folder)

            def get_degraded_image(gt):
                degraded = filter_image(gt, kernel)
                noise = np.random.normal(0.0, sigma_d, degraded.shape).astype(np.float32)
                subsampling_mask = np.zeros(degraded.shape)
                subsampling_mask[::downsampling_rate,::downsampling_rate] = 1
                degraded = subsampling_mask*(degraded + noise)
                return degraded, subsampling_mask


            for fn in file_paths:
                gt = np.array(Image.open(join(data_folder, fn)), dtype='float32')
                gt = pad_image(gt, kernel.shape)
                degraded, subsampling_mask = get_degraded_image(gt)

                print(fn)

                restored = DMSP_restore(degraded=degraded,
                                        kernel=kernel,
                                        subsampling_mask=subsampling_mask,
                                        sigma_d=sigma_d,
                                        params=params)

                restored = pad_image_rm(restored, kernel.shape)

                # saving results
                img_restored = Image.fromarray(np.clip(restored, 0, 255).astype(dtype=np.uint8))
                img_restored.save(join(out_folder, fn),"png")


In [ ]:
# Evaluate and print the results (PSNR; SSIM) per each kernel

from os import walk
from os.path import join
from DMSPRestore import compute_PSNR_pad, compute_SSIM_pad

sigma_d = 0.0
downsampling_rate = 4
for kernel_name in k_nanmes:
    out_folder = 'data/results/' + dataset_name + '_' + kernel_name + '_' + str(downsampling_rate) + '_' + str(sigma_d)
    psnrs = []
    ssims = []
    for fn in file_paths:

        gt = np.array(Image.open(join(data_folder, fn)), dtype='float32')
        rs = np.array(Image.open(join(out_folder, fn)), dtype='float32')

        psnrs += [compute_PSNR_pad(gt, rs, downsampling_rate, downsampling_rate)]
        ssims += [compute_SSIM_pad(gt, rs, downsampling_rate, downsampling_rate)]

    print(np.array(psnrs).mean())
    print(np.array(ssims).mean())